# 19장 파이프라인

> 파이프라인 기능을 알아봅니다.
- author: "Chansung Park"
- toc: true
- comments: true
- permalink: /chapter19/
- badges: true
- sticky_rank: 19

In [1]:
#hide

!pip install tpot==0.9.5
!pip install dtreeviz==0.8.2
!pip install xgbfir
!pip install yellowbrick==0.9
!pip install lime
!pip install treeinterpreter
!pip install pdpbox
!pip install shap==0.25.2
!pip install scprep
!pip install phate==0.4.2
!pip install pydotplus
!pip install scikit-plot==0.3.7
!pip install scikit-learn==0.21.0
!pip install pandas==0.23.4

     |████████████████████████████████| 901kB 5.5MB/s 
     |████████████████████████████████| 163kB 9.3MB/s 
  Created wheel for tpot: filename=TPOT-0.9.5-cp37-none-any.whl size=70870 sha256=09fddeb41412886468eeaf541ec61db7c34cf9ac48a2b8903274d14254fdf5ef
  Stored in directory: /root/.cache/pip/wheels/97/86/27/698a1366f854d2344fd0f7ee9ef57c8044ec02f182c8a0431e
  Created wheel for stopit: filename=stopit-1.1.2-cp37-none-any.whl size=11954 sha256=a2819cfc4385a4eaaf1ad81ea544ae9f7ac35691f887223ba1d5dccd4d2e4b55
  Stored in directory: /root/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7
Successfully built tpot stopit
  Created wheel for dtreeviz: filename=dtreeviz-0.8.2-cp37-none-any.whl size=24505 sha256=990dd39188e006f5adc45a05e14672201ce04cc6fb33cf44f1ef410ec5feed22
  Stored in directory: /root/.cache/pip/wheels/a2/da/b1/3178bb2b10647b5a6340c1979987e3b0c3b993cb674b0a3dcc
Successfully built dtreeviz
     |████████████████████████████████| 153kB 4.2MB/s 
  

     |████████████████████████████████| 276kB 5.5MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp37-none-any.whl size=283846 sha256=fb4cf8a829a5178681cf9a2d4349ecc3974be55c0535b1d8b9c07ce89a9d7ec4
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
Successfully built lime
     |████████████████████████████████| 57.7MB 115kB/s 
  Created wheel for pdpbox: filename=PDPbox-0.2.0-cp37-none-any.whl size=57690722 sha256=4625ea77fbd537a859131063f0bf0a3cb2a44a424be88eb32f246994efa4746c
  Stored in directory: /root/.cache/pip/wheels/7d/08/51/63fd122b04a2c87d780464eeffb94867c75bd96a64d500a3fe
Successfully built pdpbox
     |████████████████████████████████| 204kB 4.3MB/s 
  Created wheel for shap: filename=shap-0.25.2-cp37-cp37m-linux_x86_64.whl size=267716 sha256=152efc40cf1dc72104e5c85970cc1e8afa63254f7b448f726a2028125b4e818f
  Stored in directory: /root/.cache/pip/wheels/3f/cb/58/2482f534eb1c1f9ac93653887aee5b6ea83718ef041c1310bd


In [1]:
#hide

import warnings
warnings.filterwarnings('ignore')

# 19.1 분류 파이프라인

In [3]:
import pandas as pd
from sklearn.experimental import (
    enable_iterative_imputer,
)
from sklearn import (
    ensemble,
    impute,
    model_selection,    
    preprocessing,
    tree,
)
from sklearn.base import (
    BaseEstimator,
    TransformerMixin,
)
from sklearn.ensemble import (
    RandomForestClassifier,
)
from sklearn.pipeline import Pipeline
def tweak_titanic(df):
    df = df.drop(
        columns=[
            "name",
            "ticket",
            "home.dest",
            "boat",
            "body",
            "cabin",
        ]
    ).pipe(pd.get_dummies, drop_first=True)
    return df

class TitanicTransformer(
    BaseEstimator, TransformerMixin
):
    def transform(self, X):
        # assumes X is output
        # from reading Excel file
        X = tweak_titanic(X)
        X = X.drop(columns="survived")
        return X
    def fit(self, X, y):
        return self
pipe = Pipeline(
    [
        ("titan", TitanicTransformer()),
        ("impute", impute.IterativeImputer()),
        (
            "std",
            preprocessing.StandardScaler(),
        ),
        ("rf", RandomForestClassifier()),
    ]
)

In [6]:
from sklearn.model_selection import (
    train_test_split,
)

url = "https://biostat.app.vumc.org/wiki/pub/Main/DataSets/titanic3.xls"

df = pd.read_excel(url)
orig_df = df

In [7]:
from sklearn.model_selection import (
  train_test_split,
)

X_train2, X_test2, y_train2, y_test2 = train_test_split(
  orig_df,
  orig_df.survived,
  test_size=0.3,
  random_state=42,
)

pipe.fit(X_train2, y_train2)
pipe.score(X_test2, y_test2)

0.7938931297709924

In [8]:
params = {
  "rf__max_features": [0.4, "auto"],
  "rf__n_estimators": [15, 200],
}

grid = model_selection.GridSearchCV(
    pipe, cv=3, param_grid=params
)

grid.fit(orig_df, orig_df.survived)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('titan', TitanicTransformer()),
                                       ('impute',
                                        IterativeImputer(add_indicator=False,
                                                         estimator=None,
                                                         imputation_order='ascending',
                                                         initial_strategy='mean',
                                                         max_iter=10,
                                                         max_value=None,
                                                         min_value=None,
                                                         missing_values=nan,
                                                         n_nearest_features=None,
                                                         random_state=None,
          

In [9]:
grid.best_params_

{'rf__max_features': 'auto', 'rf__n_estimators': 200}

In [11]:
pipe.fit(X_train2, y_train2)
pipe.score(X_test2, y_test2)

0.7608142493638677

In [13]:
from sklearn import metrics

metrics.roc_auc_score(
  y_test2, pipe.predict(X_test2)
)

0.7429601648351648

# 19.2 회귀 파이프라인

In [16]:
#hide

import pandas as pd
from sklearn.datasets import load_boston
from sklearn import (
  model_selection,
  preprocessing,
)
b = load_boston()
bos_X = pd.DataFrame(
  b.data, columns=b.feature_names
)
bos_y = b.target

bos_X_train, bos_X_test, bos_y_train, bos_y_test = model_selection.train_test_split(
  bos_X,
  bos_y,
  test_size=0.3,
  random_state=42,
)

bos_sX = preprocessing.StandardScaler().fit_transform(bos_X)
bos_sX_train, bos_sX_test, bos_sy_train, bos_sy_test = model_selection.train_test_split(
  bos_sX,
  bos_y,
  test_size=0.3,
  random_state=42,
)

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

reg_pipe = Pipeline(
  [
    (
      "std",
      preprocessing.StandardScaler(),
    ),
    ("lr", LinearRegression()),
  ]
)

reg_pipe.fit(bos_X_train, bos_y_train)
reg_pipe.score(bos_X_test, bos_y_test)

0.7112260057484932

In [18]:
reg_pipe.named_steps["lr"].intercept_

23.01581920903956

In [19]:
reg_pipe.named_steps["lr"].coef_

array([-1.10834602,  0.80843998,  0.34313466,  0.81386426, -1.79804295,
        2.913858  , -0.29893918, -2.94251148,  2.09419303, -1.44706731,
       -2.05232232,  1.02375187, -3.88579002])

In [20]:
metrics.mean_squared_error(
  bos_y_test, reg_pipe.predict(bos_X_test)
)

21.517444231177215

# 19.3 PCA 파이프라인

In [25]:
#hide

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from sklearn.experimental import (
    enable_iterative_imputer,
)
from sklearn import (
    ensemble,
    impute,
    model_selection,    
    preprocessing,
    tree,
)
from sklearn.ensemble import (
    RandomForestClassifier,
)
from yellowbrick.model_selection import (
    ValidationCurve,
)

url = "https://biostat.app.vumc.org/wiki/pub/Main/DataSets/titanic3.xls"

df = pd.read_excel(url)
def tweak_titanic(df):
    df = df.drop(
        columns=[
            "name",
            "ticket",
            "home.dest",
            "boat",
            "body",
            "cabin",
        ]
    ).pipe(pd.get_dummies, drop_first=True)
    return df

def get_train_test_X_y(
    df, y_col, size=0.3, std_cols=None
):
    y = df[y_col]
    X = df.drop(columns=y_col)
    X_train, X_test, y_train, y_test = model_selection.train_test_split(
        X, y, test_size=size, random_state=42
    )
    cols = X.columns
    num_cols = [
        "pclass",
        "age",
        "sibsp",
        "parch",
        "fare",
    ]
    fi = impute.IterativeImputer()

    fitted = fi.fit_transform(X_train[num_cols])
    X_train = X_train.assign(**{c:fitted[:,i] for i, c in enumerate(num_cols)})
    test_fit = fi.transform(X_test[num_cols])
    X_test = X_test.assign(**{c:test_fit[:,i] for i, c in enumerate(num_cols)})
    if std_cols:
        std = preprocessing.StandardScaler()
        fitted = std.fit_transform(X_train[std_cols])
        X_train = X_train.assign(**{c:fitted[:,i] for i, c in enumerate(std_cols)})
        test_fit = std.transform(X_test[std_cols])
        X_test = X_test.assign(**{c:test_fit[:,i] for i, c in enumerate(std_cols)})

    return X_train, X_test, y_train, y_test

ti_df = tweak_titanic(df)
std_cols = "pclass,age,sibsp,fare".split(",")
X_train, X_test, y_train, y_test = get_train_test_X_y(
    ti_df, "survived", std_cols=std_cols
)

X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

In [26]:
from sklearn.decomposition import PCA


pca_pipe = Pipeline(
  [
    (
      "std",
      preprocessing.StandardScaler(),
    ),
    ("pca", PCA()),
  ]
)

X_pca = pca_pipe.fit_transform(X)

In [27]:
pca_pipe.named_steps["pca"].explained_variance_ratio_

array([0.23922833, 0.21616853, 0.1923158 , 0.10464906, 0.08154797,
       0.0727221 , 0.05130716, 0.04206107])

In [28]:
pca_pipe.named_steps["pca"].components_[0]

array([-0.63274156,  0.39602149,  0.00653646,  0.11500362,  0.5815031 ,
       -0.19764926, -0.20422289, -0.10304598])